В данном ноутбуке я генерирую заголовки для трейновой часи датасета. Имплементация модели взята из этой статьи на хабре:
https://habr.com/ru/post/581932/ 

In [3]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 32.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 35.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.9 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm
from google.colab import drive

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
drive.mount('/content/drive/')
train_data = pd.read_csv('/content/drive/MyDrive/Sber/train_data.csv', index_col=0)

Mounted at /content/drive/


In [9]:
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-multitask")
model = model.to(device)

In [10]:
def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = inputs.to(device)
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [12]:
texts = list(train_data['sentence'])
titles = []
for text in tqdm(texts):
    titles.append(generate('headline | ' + text))

100%|██████████| 1669/1669 [09:42<00:00,  2.87it/s]


In [15]:
train_data['title'] = titles
train_data.to_csv('tran_titles.csv')